In [1]:
                                                                                                                                                    # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/protein-localization/sample_submission.csv
/kaggle/input/protein-localization/field_descriptions.txt
/kaggle/input/protein-localization/protein_interactions.csv
/kaggle/input/protein-localization/train.csv
/kaggle/input/protein-localization/test.csv
/kaggle/input/protein-localization/label_legend.csv


In [2]:
train_df=pd.read_csv('../input/protein-localization/train.csv',header=None)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (444) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import io
labels=[]
file=open("../input/protein-localization/field_descriptions.txt",'r')
for i in file:
    labels.append(i.split("\t")[0].strip(":"))

In [4]:
train_df.columns=labels
train_df.head()

Protein      ESSENTIAL CLASS Actin related proteins       \
0  P239476  Non-Essential                                No   
1  P234427  Non-Essential                                No   
2  P234429  Non-Essential                                No   
3  P234430  Non-Essential                                No   
4  P239467      Essential                                No   

  CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                  No             No            No            No   
1                  No             No            No            No   
2                  No             No           Yes            No   
3                  No             No            No            No   
4                  No             No            No            No   

  CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                   No                         No   
1                   No                         No   
2                   No                         No   
3                   No                         No   
4                   No                         No   

  CLASS GTP/GDP dissociation inhibitors (GDIs)  ... FUNCTION ENERGY  \
0                                           No  ...              No   
1                                           No  ...              No   
2                                           No  ...              No   
3                                           No  ...              No   
4                                           No  ...              No   

  FUNCTION IONIC HOMEOSTASIS FUNCTION METABOLISM FUNCTION PROTEIN DESTINATION  \
0                         No                 Yes                           No   
1                         No                  No                           No   
2                         No                  No                           No   
3                        Yes                  No                           No   
4                         No                  No                           No   

  FUNCTION PROTEIN SYNTHESIS FUNCTION TRANSCRIPTION  \
0                         No                    Yes   
1                         No                    Yes   
2                         No                     No   
3                         No                    Yes   
4                         No                    Yes   

  FUNCTION TRANSPORT FACILITATION  \
0                              No   
1                              No   
2                              No   
3                              No   
4                              No   

  FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS LOCALIZATION Label  
0                                                 No             nucleus     0  
1                                                 No             nucleus     0  
2                                                 No             nucleus     0  
3                                                 No             nucleus     0  
4                                                 No             nucleus     0  

[5 rows x 2961 columns]

In [5]:
len(train_df[train_df['ESSENTIAL']=="?"])

44

In [6]:
# cols that have only "?" values
cols_train=[]
for co in train_df.columns:
        if (set(train_df[co].values)==set(['?'])):
            cols_train.append(co)

In [7]:
train_df.drop(cols_train,axis=1,inplace=True)

In [8]:
train_df.head()

Protein      ESSENTIAL CLASS Actin related proteins       \
0  P239476  Non-Essential                                No   
1  P234427  Non-Essential                                No   
2  P234429  Non-Essential                                No   
3  P234430  Non-Essential                                No   
4  P239467      Essential                                No   

  CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                  No             No            No            No   
1                  No             No            No            No   
2                  No             No           Yes            No   
3                  No             No            No            No   
4                  No             No            No            No   

  CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                   No                         No   
1                   No                         No   
2                   No                         No   
3                   No                         No   
4                   No                         No   

  CLASS GTP/GDP dissociation inhibitors (GDIs)  ... FUNCTION ENERGY  \
0                                           No  ...              No   
1                                           No  ...              No   
2                                           No  ...              No   
3                                           No  ...              No   
4                                           No  ...              No   

  FUNCTION IONIC HOMEOSTASIS FUNCTION METABOLISM FUNCTION PROTEIN DESTINATION  \
0                         No                 Yes                           No   
1                         No                  No                           No   
2                         No                  No                           No   
3                        Yes                  No                           No   
4                         No                  No                           No   

  FUNCTION PROTEIN SYNTHESIS FUNCTION TRANSCRIPTION  \
0                         No                    Yes   
1                         No                    Yes   
2                         No                     No   
3                         No                    Yes   
4                         No                    Yes   

  FUNCTION TRANSPORT FACILITATION  \
0                              No   
1                              No   
2                              No   
3                              No   
4                              No   

  FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS LOCALIZATION Label  
0                                                 No             nucleus     0  
1                                                 No             nucleus     0  
2                                                 No             nucleus     0  
3                                                 No             nucleus     0  
4                                                 No             nucleus     0  

[5 rows x 2199 columns]

In [9]:
for i in range(len(train_df)):
    if train_df.loc[i,'ESSENTIAL']=='?':
        train_df.loc[i,'ESSENTIAL']=train_df['ESSENTIAL'].mode()[0]
train_df['ESSENTIAL']

0      Non-Essential
1      Non-Essential
2      Non-Essential
3      Non-Essential
4          Essential
           ...      
857    Non-Essential
858    Non-Essential
859    Non-Essential
860        Essential
861        Essential
Name: ESSENTIAL, Length: 862, dtype: object

In [10]:
len(train_df[train_df['ESSENTIAL']=="?"])

0

In [11]:
train_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P239476  Non-Essential                                No   
1    P234427  Non-Essential                                No   
2    P234429  Non-Essential                                No   
3    P234430  Non-Essential                                No   
4    P239467      Essential                                No   
..       ...            ...                               ...   
857  P235892  Non-Essential                                No   
858  P235906  Non-Essential                                No   
859  P235897  Non-Essential                                No   
860  P235905      Essential                                No   
861  P235899      Essential                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No            No            No   
1                    No             No            No            No   
2                    No             No           Yes            No   
3                    No             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
857                  No             No            No            No   
858                  No             No            No            No   
859                  No             No            No            No   
860                  No             No            No            No   
861                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
857                   No                         No   
858                   No                         No   
859                   No                         No   
860                   No                         No   
861                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ... FUNCTION ENERGY  \
0                                             No  ...              No   
1                                             No  ...              No   
2                                             No  ...              No   
3                                             No  ...              No   
4                                             No  ...              No   
..                                           ...  ...             ...   
857                                           No  ...              No   
858                                           No  ...              No   
859                                           No  ...              No   
860                                           No  ...              No   
861                                           No  ...              No   

    FUNCTION IONIC HOMEOSTASIS FUNCTION METABOLISM  \
0                           No                 Yes   
1                           No                  No   
2                           No                  No   
3                          Yes                  No   
4                           No                  No   
..                         ...                 ...   
857                         No                  No   
858                         No                  No   
859                         No                  No   
860                         No                  No   
861                         No                 Yes   

    FUNCTION PROTEIN DESTINATION FUNCTION PROTEIN SYNTHESIS  \
0                             No                         No   
1                             No                         No   
2      

In [12]:
train_labels = train_df['Label']
train_df.drop(['Label'],axis=1,inplace=True)
train_df.drop(['LOCALIZATION'],axis=1,inplace=True)

In [13]:
train_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P239476  Non-Essential                                No   
1    P234427  Non-Essential                                No   
2    P234429  Non-Essential                                No   
3    P234430  Non-Essential                                No   
4    P239467      Essential                                No   
..       ...            ...                               ...   
857  P235892  Non-Essential                                No   
858  P235906  Non-Essential                                No   
859  P235897  Non-Essential                                No   
860  P235905      Essential                                No   
861  P235899      Essential                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No            No            No   
1                    No             No            No            No   
2                    No             No           Yes            No   
3                    No             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
857                  No             No            No            No   
858                  No             No            No            No   
859                  No             No            No            No   
860                  No             No            No            No   
861                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
857                   No                         No   
858                   No                         No   
859                   No                         No   
860                   No                         No   
861                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                             No  ...   
1                                             No  ...   
2                                             No  ...   
3                                             No  ...   
4                                             No  ...   
..                                           ...  ...   
857                                           No  ...   
858                                           No  ...   
859                                           No  ...   
860                                           No  ...   
861                                           No  ...   

    FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle)  \
0                                                  Yes                                       
1                                                  Yes                                       
2                                                  Yes                                       
3                                                  Yes                                       
4                                                  Yes                                       
..                                                 ...                                       
857                                                Yes                                       
858                                                Yes                                       
859                                                Yes                                       
860                                                Yes                                       
861

In [14]:
train_labels

0      0
1      0
2      0
3      0
4      0
      ..
857    3
858    8
859    1
860    3
861    1
Name: Label, Length: 862, dtype: int64

In [15]:
test_df=pd.read_csv('../input/protein-localization/test.csv',header=None)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (524,720,768,1112,1182,1288,1302,1352,1354,1378,1434,1436,1488,1502,1504,1604,1608,1734,1838,1908,1914,1942,1996,2246,2270,2328,2460,2514,2576,2620,2724,2758,2930) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
test_df

0              1    2    3    4    5    6    7    8    9     ... 2950  \
0    P234273      Essential   No   No   No  Yes   No   No   No   No  ...    ?   
1    P234490  Non-Essential   No   No   No   No   No   No   No   No  ...    ?   
2    P236380              ?   No   No   No   No   No   No   No   No  ...    ?   
3    P234394      Essential   No  Yes   No   No   No   No   No   No  ...    ?   
4    P234385  Non-Essential   No   No   No   No   No   No   No   No  ...    ?   
..       ...            ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
376  P235517      Essential   No   No   No   No   No   No   No   No  ...    ?   
377  P239167  Non-Essential   No   No   No   No   No   No   No   No  ...    ?   
378  P240327  Non-Essential   No   No   No   No   No   No   No   No  ...    ?   
379  P235928      Essential   No   No   No   No   No   No   No   No  ...    ?   
380  P235898              ?   No   No   No   No   No   No   No   No  ...    ?   

    2951 2952 2953 2954 2955 2956 2957 2958 2959  
0      ?    ?    ?    ?    ?    ?    ?    ?    ?  
1      ?    ?    ?    ?    ?    ?    ?    ?    ?  
2      ?    ?    ?    ?    ?    ?    ?    ?    ?  
3      ?    ?    ?    ?    ?    ?    ?    ?    ?  
4      ?    ?    ?    ?    ?    ?    ?    ?    ?  
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  
376    ?    ?    ?    ?    ?    ?    ?    ?    ?  
377    ?    ?    ?    ?    ?    ?    ?    ?    ?  
378    ?    ?    ?    ?    ?    ?    ?    ?    ?  
379    ?    ?    ?    ?    ?    ?    ?    ?    ?  
380    ?    ?    ?    ?    ?    ?    ?    ?    ?  

[381 rows x 2960 columns]

In [17]:
labels2 = [] 
labels2 = labels
labels2.remove("Label")
len(labels2)

2960

In [18]:
test_df.columns=labels2
test_df.head()

Protein      ESSENTIAL CLASS Actin related proteins       \
0  P234273      Essential                                No   
1  P234490  Non-Essential                                No   
2  P236380              ?                                No   
3  P234394      Essential                                No   
4  P234385  Non-Essential                                No   

  CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                  No             No           Yes            No   
1                  No             No            No            No   
2                  No             No            No            No   
3                 Yes             No            No            No   
4                  No             No            No            No   

  CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                   No                         No   
1                   No                         No   
2                   No                         No   
3                   No                         No   
4                   No                         No   

  CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                           No  ...   
1                                           No  ...   
2                                           No  ...   
3                                           No  ...   
4                                           No  ...   

  FUNCTION CELLULAR TRANSPORT AND TRANSPORTMECHANISMS FUNCTION ENERGY  \
0                                                  ?                ?   
1                                                  ?                ?   
2                                                  ?                ?   
3                                                  ?                ?   
4                                                  ?                ?   

  FUNCTION IONIC HOMEOSTASIS FUNCTION METABOLISM FUNCTION PROTEIN DESTINATION  \
0                          ?                   ?                            ?   
1                          ?                   ?                            ?   
2                          ?                   ?                            ?   
3                          ?                   ?                            ?   
4                          ?                   ?                            ?   

  FUNCTION PROTEIN SYNTHESIS FUNCTION TRANSCRIPTION  \
0                          ?                      ?   
1                          ?                      ?   
2                          ?                      ?   
3                          ?                      ?   
4                          ?                      ?   

  FUNCTION TRANSPORT FACILITATION  \
0                               ?   
1                               ?   
2                               ?   
3                               ?   
4                               ?   

  FUNCTION TRANSPOSABLE ELEMENTS VIRAL AND PLASMID PROTEINS LOCALIZATION  
0                                                  ?                   ?  
1                                                  ?                   ?  
2                                                  ?                   ?  
3                                                  ?                   ?  
4                                                  ?                   ?  

[5 rows x 2960 columns]

In [19]:
test_df = test_df.iloc[: , :-1]

In [20]:
test_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P234273      Essential                                No   
1    P234490  Non-Essential                                No   
2    P236380              ?                                No   
3    P234394      Essential                                No   
4    P234385  Non-Essential                                No   
..       ...            ...                               ...   
376  P235517      Essential                                No   
377  P239167  Non-Essential                                No   
378  P240327  Non-Essential                                No   
379  P235928      Essential                                No   
380  P235898              ?                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No           Yes            No   
1                    No             No            No            No   
2                    No             No            No            No   
3                   Yes             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
376                  No             No            No            No   
377                  No             No            No            No   
378                  No             No            No            No   
379                  No             No            No            No   
380                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
376                   No                         No   
377                   No                         No   
378                   No                         No   
379                   No                         No   
380                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                             No  ...   
1                                             No  ...   
2                                             No  ...   
3                                             No  ...   
4                                             No  ...   
..                                           ...  ...   
376                                           No  ...   
377                                           No  ...   
378                                           No  ...   
379                                           No  ...   
380                                           No  ...   

    FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle)  \
0                                                    ?                                       
1                                                    ?                                       
2                                                    ?                                       
3                                                    ?                                       
4                                                    ?                                       
..                                                 ...                                       
376                                                  ?                                       
377                                                  ?                                       
378                                                  ?                                       
379                                                  ?                                       
380

In [21]:
test_df.drop(cols_train,axis=1,inplace=True)

In [22]:
test_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P234273      Essential                                No   
1    P234490  Non-Essential                                No   
2    P236380              ?                                No   
3    P234394      Essential                                No   
4    P234385  Non-Essential                                No   
..       ...            ...                               ...   
376  P235517      Essential                                No   
377  P239167  Non-Essential                                No   
378  P240327  Non-Essential                                No   
379  P235928      Essential                                No   
380  P235898              ?                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No           Yes            No   
1                    No             No            No            No   
2                    No             No            No            No   
3                   Yes             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
376                  No             No            No            No   
377                  No             No            No            No   
378                  No             No            No            No   
379                  No             No            No            No   
380                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
376                   No                         No   
377                   No                         No   
378                   No                         No   
379                   No                         No   
380                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                             No  ...   
1                                             No  ...   
2                                             No  ...   
3                                             No  ...   
4                                             No  ...   
..                                           ...  ...   
376                                           No  ...   
377                                           No  ...   
378                                           No  ...   
379                                           No  ...   
380                                           No  ...   

    FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle)  \
0                                                    ?                                       
1                                                    ?                                       
2                                                    ?                                       
3                                                    ?                                       
4                                                    ?                                       
..                                                 ...                                       
376                                                  ?                                       
377                                                  ?                                       
378                                                  ?                                       
379                                                  ?                                       
380

In [23]:
#to remove all rows in essential that have "?" in test
for i in range(len(test_df)):
    if test_df.loc[i,'ESSENTIAL']=='?':
        test_df.loc[i,'ESSENTIAL']=test_df['ESSENTIAL'].mode()[0]
test_df['ESSENTIAL']

0          Essential
1      Non-Essential
2      Non-Essential
3          Essential
4      Non-Essential
           ...      
376        Essential
377    Non-Essential
378    Non-Essential
379        Essential
380    Non-Essential
Name: ESSENTIAL, Length: 381, dtype: object

In [24]:
len(test_df[test_df['ESSENTIAL']=="?"])

0

In [25]:
test_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P234273      Essential                                No   
1    P234490  Non-Essential                                No   
2    P236380  Non-Essential                                No   
3    P234394      Essential                                No   
4    P234385  Non-Essential                                No   
..       ...            ...                               ...   
376  P235517      Essential                                No   
377  P239167  Non-Essential                                No   
378  P240327  Non-Essential                                No   
379  P235928      Essential                                No   
380  P235898  Non-Essential                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No           Yes            No   
1                    No             No            No            No   
2                    No             No            No            No   
3                   Yes             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
376                  No             No            No            No   
377                  No             No            No            No   
378                  No             No            No            No   
379                  No             No            No            No   
380                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
376                   No                         No   
377                   No                         No   
378                   No                         No   
379                   No                         No   
380                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                             No  ...   
1                                             No  ...   
2                                             No  ...   
3                                             No  ...   
4                                             No  ...   
..                                           ...  ...   
376                                           No  ...   
377                                           No  ...   
378                                           No  ...   
379                                           No  ...   
380                                           No  ...   

    FUNCTION CELLULAR ORGANIZATION (proteins are localized to the corresponding organelle)  \
0                                                    ?                                       
1                                                    ?                                       
2                                                    ?                                       
3                                                    ?                                       
4                                                    ?                                       
..                                                 ...                                       
376                                                  ?                                       
377                                                  ?                                       
378                                                  ?                                       
379                                                  ?                                       
380

In [26]:
#to remove all columns that have "?" in test
cols_test=[]
for co in test_df.columns:
        if (set(test_df[co].values)==set(['?'])):
            cols_test.append(co)

In [27]:
test_df.drop(cols_test,axis=1,inplace=True)

In [28]:
test_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P234273      Essential                                No   
1    P234490  Non-Essential                                No   
2    P236380  Non-Essential                                No   
3    P234394      Essential                                No   
4    P234385  Non-Essential                                No   
..       ...            ...                               ...   
376  P235517      Essential                                No   
377  P239167  Non-Essential                                No   
378  P240327  Non-Essential                                No   
379  P235928      Essential                                No   
380  P235898  Non-Essential                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No           Yes            No   
1                    No             No            No            No   
2                    No             No            No            No   
3                   Yes             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
376                  No             No            No            No   
377                  No             No            No            No   
378                  No             No            No            No   
379                  No             No            No            No   
380                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
376                   No                         No   
377                   No                         No   
378                   No                         No   
379                   No                         No   
380                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                             No  ...   
1                                             No  ...   
2                                             No  ...   
3                                             No  ...   
4                                             No  ...   
..                                           ...  ...   
376                                           No  ...   
377                                           No  ...   
378                                           No  ...   
379                                           No  ...   
380                                           No  ...   

    INTERACTING PROTEIN P235892 Type INTERACTING PROTEIN P235892 Corr  \
0                                 No                              0.0   
1                                 No                              0.0   
2                                 No                              0.0   
3                                 No                              0.0   
4                                 No                              0.0   
..                               ...                              ...   
376                               No                              0.0   
377                               No                              0.0   
378                               No                              0.0   
379                               No                              0.0   
380                               No                              0.0   

    INTERACTING PROTEIN P235906 Type INTERACTING PROTEIN P235906 Corr  \
0                                 No                              0.0   
1             

In [29]:
# removing the same columns from test in train as well
train_df.drop(cols_test,axis=1,inplace=True)

In [30]:
train_df

Protein      ESSENTIAL CLASS Actin related proteins       \
0    P239476  Non-Essential                                No   
1    P234427  Non-Essential                                No   
2    P234429  Non-Essential                                No   
3    P234430  Non-Essential                                No   
4    P239467      Essential                                No   
..       ...            ...                               ...   
857  P235892  Non-Essential                                No   
858  P235906  Non-Essential                                No   
859  P235897  Non-Essential                                No   
860  P235905      Essential                                No   
861  P235899      Essential                                No   

    CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                    No             No            No            No   
1                    No             No            No            No   
2                    No             No           Yes            No   
3                    No             No            No            No   
4                    No             No            No            No   
..                  ...            ...           ...           ...   
857                  No             No            No            No   
858                  No             No            No            No   
859                  No             No            No            No   
860                  No             No            No            No   
861                  No             No            No            No   

    CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                     No                         No   
1                     No                         No   
2                     No                         No   
3                     No                         No   
4                     No                         No   
..                   ...                        ...   
857                   No                         No   
858                   No                         No   
859                   No                         No   
860                   No                         No   
861                   No                         No   

    CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                             No  ...   
1                                             No  ...   
2                                             No  ...   
3                                             No  ...   
4                                             No  ...   
..                                           ...  ...   
857                                           No  ...   
858                                           No  ...   
859                                           No  ...   
860                                           No  ...   
861                                           No  ...   

    INTERACTING PROTEIN P235892 Type INTERACTING PROTEIN P235892 Corr  \
0                                 No                              0.0   
1                                 No                              0.0   
2                                 No                              0.0   
3                                 No                              0.0   
4                                 No                              0.0   
..                               ...                              ...   
857                               No                              0.0   
858                               No                              0.0   
859                               No                              0.0   
860                               No                              0.0   
861                               No                              0.0   

    INTERACTING PROTEIN P235906 Type INTERACTING PROTEIN P235906 Corr  \
0                                 No                                0   
1             

In [31]:
# combining train and test to do one hot encoding since string columns are present
df_new = pd.concat([train_df,test_df],ignore_index=True)
df_new

Protein      ESSENTIAL CLASS Actin related proteins       \
0     P239476  Non-Essential                                No   
1     P234427  Non-Essential                                No   
2     P234429  Non-Essential                                No   
3     P234430  Non-Essential                                No   
4     P239467      Essential                                No   
...       ...            ...                               ...   
1238  P235517      Essential                                No   
1239  P239167  Non-Essential                                No   
1240  P240327  Non-Essential                                No   
1241  P235928      Essential                                No   
1242  P235898  Non-Essential                                No   

     CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                     No             No            No            No   
1                     No             No            No            No   
2                     No             No           Yes            No   
3                     No             No            No            No   
4                     No             No            No            No   
...                  ...            ...           ...           ...   
1238                  No             No            No            No   
1239                  No             No            No            No   
1240                  No             No            No            No   
1241                  No             No            No            No   
1242                  No             No            No            No   

     CLASS Dehydrogenases CLASS GTP-binding proteins  \
0                      No                         No   
1                      No                         No   
2                      No                         No   
3                      No                         No   
4                      No                         No   
...                   ...                        ...   
1238                   No                         No   
1239                   No                         No   
1240                   No                         No   
1241                   No                         No   
1242                   No                         No   

     CLASS GTP/GDP dissociation inhibitors (GDIs)  ...  \
0                                              No  ...   
1                                              No  ...   
2                                              No  ...   
3                                              No  ...   
4                                              No  ...   
...                                           ...  ...   
1238                                           No  ...   
1239                                           No  ...   
1240                                           No  ...   
1241                                           No  ...   
1242                                           No  ...   

     INTERACTING PROTEIN P235892 Type INTERACTING PROTEIN P235892 Corr  \
0                                  No                              0.0   
1                                  No                              0.0   
2                                  No                              0.0   
3                                  No                              0.0   
4                                  No                              0.0   
...                               ...                              ...   
1238                               No                              0.0   
1239                               No                              0.0   
1240                               No                              0.0   
1241                               No                              0.0   
1242                               No                              0.0   

     INTERACTING PROTEIN P235906 Type INTERACTING PROTEIN P235906 Corr  \
0                          

In [32]:
# removing columns based on correlation
cols_to_remove = []
df_new_corr = df_new.corr()

In [33]:
df_new_corr

"NUM INTERACTING WITH FUNCTION ""CELL GROWTH"  \
"NUM INTERACTING WITH FUNCTION ""CELL GROWTH"                                            1.000000   
"NUM INTERACTING WITH FUNCTION ""CELL RESCUE"                                            0.277985   
NUM INTERACTING WITH FUNCTION CELLULAR BIOGENES...                                       0.283160   
NUM INTERACTING WITH FUNCTION CELLULAR COMMUNIC...                                       0.412342   
NUM INTERACTING WITH FUNCTION CELLULAR ORGANIZA...                                       0.637581   
...                                                                                           ...   
INTERACTING PROTEIN P235892 Corr                                                         0.026536   
INTERACTING PROTEIN P235906 Corr                                                        -0.015851   
INTERACTING PROTEIN P235897 Corr                                                        -0.015851   
INTERACTING PROTEIN P235905 Corr                                                              NaN   
INTERACTING PROTEIN P235899 Corr                                                        -0.026295   

                                                    "NUM INTERACTING WITH FUNCTION ""CELL RESCUE"  \
"NUM INTERACTING WITH FUNCTION ""CELL GROWTH"                                            0.277985   
"NUM INTERACTING WITH FUNCTION ""CELL RESCUE"                                            1.000000   
NUM INTERACTING WITH FUNCTION CELLULAR BIOGENES...                                       0.013546   
NUM INTERACTING WITH FUNCTION CELLULAR COMMUNIC...                                       0.307590   
NUM INTERACTING WITH FUNCTION CELLULAR ORGANIZA...                                       0.201990   
...                                                                                           ...   
INTERACTING PROTEIN P235892 Corr                                                        -0.018869   
INTERACTING PROTEIN P235906 Corr                                                        -0.008662   
INTERACTING PROTEIN P235897 Corr                                                        -0.008662   
INTERACTING PROTEIN P235905 Corr                                                              NaN   
INTERACTING PROTEIN P235899 Corr                                                         0.008662   

                                                    NUM INTERACTING WITH FUNCTION CELLULAR BIOGENESIS (proteins are not localized to the corresponding organelle)    \
"NUM INTERACTING WITH FUNCTION ""CELL GROWTH"                                                0.283160                                                                 
"NUM INTERACTING WITH FUNCTION ""CELL RESCUE"                                                0.013546                                                                 
NUM INTERACTING WITH FUNCTION CELLULAR BIOGENES...                                           1.000000                                                                 
NUM INTERACTING WITH FUNCTION CELLULAR COMMUNIC...                                           0.067905                                                                 
NUM INTERACTING WITH FUNCTION CELLULAR ORGANIZA...                                           0.274275                                                                 
...                                                                                               ...                                                                 
INTERACTING PROTEIN P235892 Corr                                                             0.027861                                                                 
INTERACTING PROTEIN P235906 Corr                                                             0.058357                                                                 
INTERACTING PROTEIN P235897 Corr                                                            -0.008498                      

In [34]:
cols_to_remove=set()
for j in df_new_corr.columns:
        if df_new_corr.loc[j,j]>0.8:
            cols_to_remove.add(j)

len(cols_to_remove)

751

In [35]:
for co in df_new.columns:
        if set(df_new[co].values)== set(['No']) or set(df_new[co].values)== set(['Yes']):
            cols_to_remove.add(co)

In [36]:
df_new.drop(cols_to_remove,axis=1,inplace=True)

In [37]:
df_new

Protein      ESSENTIAL CLASS Actin related proteins       \
0     P239476  Non-Essential                                No   
1     P234427  Non-Essential                                No   
2     P234429  Non-Essential                                No   
3     P234430  Non-Essential                                No   
4     P239467      Essential                                No   
...       ...            ...                               ...   
1238  P235517      Essential                                No   
1239  P239167  Non-Essential                                No   
1240  P240327  Non-Essential                                No   
1241  P235928      Essential                                No   
1242  P235898  Non-Essential                                No   

     CLASS Actins        CLASS Adaptins CLASS ATPases CLASS Cyclins  \
0                     No             No            No            No   
1                     No             No            No            No   
2                     No             No           Yes            No   
3                     No             No            No            No   
4                     No             No            No            No   
...                  ...            ...           ...           ...   
1238                  No             No            No            No   
1239                  No             No            No            No   
1240                  No             No            No            No   
1241                  No             No            No            No   
1242                  No             No            No            No   

     CLASS GTP-binding proteins CLASS GTP/GDP dissociation inhibitors (GDIs)  \
0                            No                                           No   
1                            No                                           No   
2                            No                                           No   
3                            No                                           No   
4                            No                                           No   
...                         ...                                          ...   
1238                         No                                           No   
1239                         No                                           No   
1240                         No                                           No   
1241                         No                                           No   
1242                         No                                           No   

     CLASS GTP/GDP-exchange factors (GEFs)  ...  \
0                                       No  ...   
1                                       No  ...   
2                                       No  ...   
3                                       No  ...   
4                                       No  ...   
...                                    ...  ...   
1238                                    No  ...   
1239                                    No  ...   
1240                                    No  ...   
1241                                    No  ...   
1242                                    No  ...   

     INTERACTING PROTEIN P235881 Type INTERACTING PROTEIN P235914 Corr  \
0                                  No                                0   
1                                  No                                0   
2                                  No                                0   
3                                  No                                0   
4                                  No                                0   
...                               ...                              ...   
1238                               No                                0   
1239                               No                                0   
1240                               No                                0   
1241                               No         

In [38]:
# one hot encoding
df_new=df_new.iloc[:,1:]
df_new=pd.get_dummies(df_new)
df_new

"NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS"  \
0                                                     0         
1                                                     0         
2                                                     0         
3                                                     0         
4                                                     0         
...                                                 ...         
1238                                                  0         
1239                                                  0         
1240                                                  0         
1241                                                  0         
1242                                                  0         

      INTERACTING PROTEIN P239476 Corr  INTERACTING PROTEIN P234437 Corr  \
0                                    0                                 0   
1                                    0                                 0   
2                                    0                                 0   
3                                    0                                 0   
4                                    0                                 0   
...                                ...                               ...   
1238                                 0                                 0   
1239                                 0                                 0   
1240                                 0                                 0   
1241                                 0                                 0   
1242                                 0                                 0   

      INTERACTING PROTEIN P239438 Corr  INTERACTING PROTEIN P234452 Corr  \
0                                    0                                 0   
1                                    0                                 0   
2                                    0                                 0   
3                                    0                                 0   
4                                    0                                 0   
...                                ...                               ...   
1238                                 0                                 0   
1239                                 0                                 0   
1240                                 0                                 0   
1241                                 0                                 0   
1242                                 0                                 0   

      INTERACTING PROTEIN P234509 Corr  INTERACTING PROTEIN P239294 Corr  \
0                                    0                               0.0   
1                                    0                               0.0   
2                                    0                               0.0   
3                                    0                               0.0   
4                                    0                               0.0   
...                                ...                               ...   
1238                                 0                               0.0   
1239                                 0                               0.0   
1240                                 0                               0.0   
1241                                 0                               0.0   
1242                                 0                               0.0   

      INTERACTING PROTEIN P239196 Corr  INTERACTING PROTEIN P239186 Corr  \
0                                    0                                 0   
1                                    0                                 0   
2                                    0                                 0   
3                                    0                                 0   
4                                    0                                 0   
...                           

In [39]:
np.any(np.isnan(df_new))

True

In [40]:
df_new = df_new.replace(np.nan, 0)
df_new

"NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS"  \
0                                                     0         
1                                                     0         
2                                                     0         
3                                                     0         
4                                                     0         
...                                                 ...         
1238                                                  0         
1239                                                  0         
1240                                                  0         
1241                                                  0         
1242                                                  0         

      INTERACTING PROTEIN P239476 Corr  INTERACTING PROTEIN P234437 Corr  \
0                                    0                                 0   
1                                    0                                 0   
2                                    0                                 0   
3                                    0                                 0   
4                                    0                                 0   
...                                ...                               ...   
1238                                 0                                 0   
1239                                 0                                 0   
1240                                 0                                 0   
1241                                 0                                 0   
1242                                 0                                 0   

      INTERACTING PROTEIN P239438 Corr  INTERACTING PROTEIN P234452 Corr  \
0                                    0                                 0   
1                                    0                                 0   
2                                    0                                 0   
3                                    0                                 0   
4                                    0                                 0   
...                                ...                               ...   
1238                                 0                                 0   
1239                                 0                                 0   
1240                                 0                                 0   
1241                                 0                                 0   
1242                                 0                                 0   

      INTERACTING PROTEIN P234509 Corr  INTERACTING PROTEIN P239294 Corr  \
0                                    0                               0.0   
1                                    0                               0.0   
2                                    0                               0.0   
3                                    0                               0.0   
4                                    0                               0.0   
...                                ...                               ...   
1238                                 0                               0.0   
1239                                 0                               0.0   
1240                                 0                               0.0   
1241                                 0                               0.0   
1242                                 0                               0.0   

      INTERACTING PROTEIN P239196 Corr  INTERACTING PROTEIN P239186 Corr  \
0                                    0                                 0   
1                                    0                                 0   
2                                    0                                 0   
3                                    0                                 0   
4                                    0                                 0   
...                           

In [41]:
np.any(np.isnan(df_new))

False

In [42]:
# getting back the original train_df but now with one hot encoding
X_train = df_new.iloc[:len(train_df),:]
X_train

"NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS"  \
0                                                    0         
1                                                    0         
2                                                    0         
3                                                    0         
4                                                    0         
..                                                 ...         
857                                                  0         
858                                                  0         
859                                                  0         
860                                                  0         
861                                                  0         

     INTERACTING PROTEIN P239476 Corr  INTERACTING PROTEIN P234437 Corr  \
0                                   0                                 0   
1                                   0                                 0   
2                                   0                                 0   
3                                   0                                 0   
4                                   0                                 0   
..                                ...                               ...   
857                                 0                                 0   
858                                 0                                 0   
859                                 0                                 0   
860                                 0                                 0   
861                                 0                                 0   

     INTERACTING PROTEIN P239438 Corr  INTERACTING PROTEIN P234452 Corr  \
0                                   0                                 0   
1                                   0                                 0   
2                                   0                                 0   
3                                   0                                 0   
4                                   0                                 0   
..                                ...                               ...   
857                                 0                                 0   
858                                 0                                 0   
859                                 0                                 0   
860                                 0                                 0   
861                                 0                                 0   

     INTERACTING PROTEIN P234509 Corr  INTERACTING PROTEIN P239294 Corr  \
0                                   0                               0.0   
1                                   0                               0.0   
2                                   0                               0.0   
3                                   0                               0.0   
4                                   0                               0.0   
..                                ...                               ...   
857                                 0                               0.0   
858                                 0                               0.0   
859                                 0                               0.0   
860                                 0                               0.0   
861                                 0                               0.0   

     INTERACTING PROTEIN P239196 Corr  INTERACTING PROTEIN P239186 Corr  \
0                                   0                                 0   
1                                   0                                 0   
2                                   0                                 0   
3                                   0                                 0   
4                                   0                                 0   
..                                ...                               ...   
857     

In [43]:
# getting back the original test_df but now with one hot encoding
X_test = df_new.iloc[len(train_df):,:]
X_test

"NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS"  \
862                                                   0         
863                                                   0         
864                                                   0         
865                                                   0         
866                                                   0         
...                                                 ...         
1238                                                  0         
1239                                                  0         
1240                                                  0         
1241                                                  0         
1242                                                  0         

      INTERACTING PROTEIN P239476 Corr  INTERACTING PROTEIN P234437 Corr  \
862                                  0                                 0   
863                                  0                                 0   
864                                  0                                 0   
865                                  0                                 0   
866                                  0                                 0   
...                                ...                               ...   
1238                                 0                                 0   
1239                                 0                                 0   
1240                                 0                                 0   
1241                                 0                                 0   
1242                                 0                                 0   

      INTERACTING PROTEIN P239438 Corr  INTERACTING PROTEIN P234452 Corr  \
862                                  0                                 0   
863                                  0                                 0   
864                                  0                                 0   
865                                  0                                 0   
866                                  0                                 0   
...                                ...                               ...   
1238                                 0                                 0   
1239                                 0                                 0   
1240                                 0                                 0   
1241                                 0                                 0   
1242                                 0                                 0   

      INTERACTING PROTEIN P234509 Corr  INTERACTING PROTEIN P239294 Corr  \
862                                  0                               0.0   
863                                  0                               0.0   
864                                  0                               0.0   
865                                  0                               0.0   
866                                  0                               0.0   
...                                ...                               ...   
1238                                 0                               0.0   
1239                                 0                               0.0   
1240                                 0                               0.0   
1241                                 0                               0.0   
1242                                 0                               0.0   

      INTERACTING PROTEIN P239196 Corr  INTERACTING PROTEIN P239186 Corr  \
862                                  0                                 0   
863                                  0                                 0   
864                                  0                                 0   
865                                  0                                 0   
866                                  0                                 0   
...                           

In [44]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862 entries, 0 to 861
Columns: 2878 entries, "NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS" to INTERACTING PROTEIN P235899 Type_No
dtypes: float64(11), int64(91), uint8(2776)
memory usage: 3.0 MB


In [45]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 862 to 1242
Columns: 2878 entries, "NUM INTERACTING WITH FUNCTION ""TRANSPOSABLE ELEMENTS" to INTERACTING PROTEIN P235899 Type_No
dtypes: float64(11), int64(91), uint8(2776)
memory usage: 1.3 MB


## Using standard Scalar here to have mean 0 and sd=1

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_train

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.03407991,  0.03407991]])

In [47]:
X_test=scaler.fit_transform(X_test)
X_test

array([[0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05129892, 0.        ,
        0.        ]])

In [48]:
train_labels

0      0
1      0
2      0
3      0
4      0
      ..
857    3
858    8
859    1
860    3
861    1
Name: Label, Length: 862, dtype: int64

In [49]:
labels_df=pd.read_csv('../input/protein-localization/label_legend.csv')

## Random Forest HyperTuning

In [50]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,20,30,40,80,100],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [5, 10,15,20],
    'n_estimators': [200, 300, 400,800]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_results=grid_search.fit(X_train,train_labels)

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


In [51]:
grid_results.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 200}

In [52]:
# rf_results.cv_results_

In [53]:
len(train_labels)

862

In [54]:
rf

RandomForestClassifier()

In [55]:
stop here

SyntaxError: invalid syntax (<ipython-input-55-a96ba3aab008>, line 1)

In [ ]:
tuned_params=grid_results.best_params_
tuned_params

In [ ]:
tuned_params['n_estimators']

# improvements after hypertuning should be below one and predict final using this and try to submit it.

In [57]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
sss = KFold(n_splits=10,shuffle=True,random_state=0)

epochs=0
for train_index, val_index in sss.split(X_train, train_labels):
    #print("TRAIN:", train_index, "Val:", val_index)
    X_train_, X_val = X_train[train_index], X_train[val_index]
    y_train_, y_val = train_labels[train_index], train_labels[val_index]
    #grid_search.fit(X_train_,y_train_)
    y_predicted=grid_search.predict(X_val)
    print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
    epochs+=1

Epochs #: 0 -----> val accuracy : 0.6666666666666666
Epochs #: 1 -----> val accuracy : 0.6781609195402298
Epochs #: 2 -----> val accuracy : 0.6744186046511628
Epochs #: 3 -----> val accuracy : 0.5813953488372093
Epochs #: 4 -----> val accuracy : 0.686046511627907
Epochs #: 5 -----> val accuracy : 0.627906976744186
Epochs #: 6 -----> val accuracy : 0.6627906976744186
Epochs #: 7 -----> val accuracy : 0.6627906976744186
Epochs #: 8 -----> val accuracy : 0.5930232558139535
Epochs #: 9 -----> val accuracy : 0.6627906976744186


In [58]:
y_pred=grid_search.predict(X_test)

In [59]:
y_pred

array([0, 0, 0, 3, 1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 7, 0, 0, 0, 0, 0,
       0, 0, 1, 7, 0, 0, 0, 0, 5, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0,
       3, 2, 0, 0, 0, 0, 0, 3, 0, 3, 3, 1, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0,
       6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 5, 1, 3, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 5, 3, 6, 0,
       1, 0, 0, 0, 3, 8, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 3, 1, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 6, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 8, 0, 1,

In [ ]:
ans_file_rf=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':y_pred})

In [ ]:
ans_file_rf

In [ ]:
ans_file_rf.to_csv('/kaggle/working/submissions_rf.csv',index=False)

In [ ]:
stop here

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
sss = KFold(n_splits=10,shuffle=True,random_state=0)

epochs=0
for train_index, val_index in sss.split(X_train, train_labels):
    #print("TRAIN:", train_index, "Val:", val_index)
    X_train_, X_val = X_train[train_index], X_train[val_index]
    y_train_, y_val = train_labels[train_index], train_labels[val_index]
    rf.fit(X_train_,y_train_)
    y_predicted=rf.predict(X_val)
    print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
    epochs+=1

In [ ]:
ans=rf.predict(X_test)

In [ ]:
ans

In [ ]:
ans_file_rf=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':ans})

In [ ]:
ans_file_rf

In [ ]:
ans_file_rf.to_csv('/kaggle/working/submissions_rf.csv',index=False)

# add classification report

In [ ]:
from skelarn.metrics import classification report

print(classification_report())

## Plot feature importance

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=grid_results.best_params_['n_estimators'], n_jobs=-1, random_state=1)
rnd_clf.fit(X_train, train_labels)

In [ ]:
import matplotlib.pyplot as plt
features = df_new.columns
importances = rnd_clf.feature_importances_
# we have loads of columns so we only display top 100 in the bar chart
indices = np.argsort(importances)
indices=indices[:-10:-1]
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
plt.savefig('/kaggle/working/books_read.png')

## Hyper-tuning the n_estimators

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [100, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_results=grid_search.fit(X_train,train_labels)

In [ ]:
grid_results.cv_results_

In [ ]:
plt.plot(param_grid['n_estimators'],grid_results.cv_results_['mean_test_score'])
plt.title('n-estimators v/s Accuracy')
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
grid_results.best_params_

## Max Depth

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
 }
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_results=grid_search.fit(X_train,train_labels)

In [ ]:
grid_results.cv_results_

In [ ]:
plt.plot(param_grid['max_depth'],grid_results.cv_results_['mean_test_score'])
plt.title('max_depth v/s Accuracy')
plt.xlabel('max-depth')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
grid_results.best_params_

# make a new rf after tuning the hyper-parameters

In [157]:
rnd_clf = RandomForestClassifier(n_estimators=200,max_depth=70, n_jobs=-1, random_state=1)

In [158]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
sss = KFold(n_splits=10,shuffle=True,random_state=0)

epochs=0
for train_index, val_index in sss.split(X_train, train_labels):
    #print("TRAIN:", train_index, "Val:", val_index)
    X_train_, X_val = X_train[train_index], X_train[val_index]
    y_train_, y_val = train_labels[train_index], train_labels[val_index]
    rnd_clf.fit(X_train_,y_train_)
    y_predicted=rnd_clf.predict(X_val)
    print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
    epochs+=1

Epochs #: 0 -----> val accuracy : 0.7011494252873564
Epochs #: 1 -----> val accuracy : 0.7126436781609196
Epochs #: 2 -----> val accuracy : 0.6976744186046512
Epochs #: 3 -----> val accuracy : 0.686046511627907
Epochs #: 4 -----> val accuracy : 0.6744186046511628
Epochs #: 5 -----> val accuracy : 0.7093023255813954
Epochs #: 6 -----> val accuracy : 0.7325581395348837
Epochs #: 7 -----> val accuracy : 0.6744186046511628
Epochs #: 8 -----> val accuracy : 0.6976744186046512
Epochs #: 9 -----> val accuracy : 0.7209302325581395


## add test predictions here:

In [159]:
ans=rnd_clf.predict(X_test)

In [160]:
ans

array([1, 0, 1, 3, 1, 1, 0, 0, 5, 7, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 7, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 7, 7, 4, 0, 0, 0, 0,
       1, 0, 1, 7, 7, 0, 7, 0, 5, 0, 0, 0, 1, 1, 4, 7, 0, 0, 0, 3, 1, 1,
       0, 5, 0, 1, 0, 5, 0, 8, 0, 3, 1, 1, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0,
       3, 2, 0, 0, 5, 0, 1, 3, 0, 3, 3, 1, 0, 1, 8, 0, 3, 1, 4, 0, 0, 1,
       6, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 7,
       0, 1, 1, 3, 0, 1, 1, 1, 0, 8, 0, 0, 2, 8, 0, 3, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 7, 3, 0, 3, 0, 5, 3, 6, 0,
       1, 8, 1, 1, 3, 8, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 3, 1, 0,
       7, 2, 1, 0, 2, 0, 2, 0, 0, 1, 0, 1, 0, 3, 1, 0, 8, 0, 5, 0, 0, 0,
       1, 7, 1, 1, 8, 7, 8, 0, 3, 0, 0, 5, 7, 1, 0, 1, 0, 0, 0, 6, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 5, 0, 1, 0, 4, 3, 2, 1, 7, 3, 1, 0, 1, 0, 0,
       2, 8, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 1, 8, 1, 1,

In [165]:
ans_file_rf=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':ans})

In [166]:
ans_file_rf.to_csv('/kaggle/working/submissions_tuned.csv',index=False)

In [ ]:
stop here

# HyperTuning the MLP now

## hidden layer sizes

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold

nn=MLPClassifier(early_stopping=True) # enter max_iter=500 and it will take less time.
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

param_grid = {
    'hidden_layer_sizes': [(10,30,),(20,),(100,20),(200,),(100,),(200,200)],
   }
    
grid=GridSearchCV(nn,param_grid=param_grid,verbose=True,cv=cv)
grid_result=grid.fit(X_train,train_labels)




In [ ]:
grid_result.cv_results_



In [ ]:
grid_result.best_params_

In [ ]:
plt.bar(["(10,30,)","(20,)","(100,20)","(200,)","(100,)","(200,200)"],grid_result.cv_results_['mean_test_score'])
plt.title('Hidden Layer size v/s Accuracy')
plt.xlabel('Hidden Layer sizes')
plt.ylabel('Accuracy')
plt.show()

## max_iter

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold

nn=MLPClassifier(early_stopping=True) # enter max_iter=500 and it will take less time.
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

param_grid = {
    'max_iter':[200,300,400,500,600,700,800,900,1000]
}
    
grid=GridSearchCV(nn,param_grid=param_grid,verbose=True,cv=cv)
grid_result=grid.fit(X_train,train_labels)




In [ ]:
grid_result.cv_results_

In [ ]:
grid_result.best_params_

In [ ]:
plt.plot(param_grid['max_iter'],grid_result.cv_results_['mean_test_score'])
plt.title('max_iter v/s Accuracy')
plt.xlabel('max-iter')
plt.ylabel('Accuracy')
plt.show()

## activation

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold

nn=MLPClassifier(max_iter=500,early_stopping=True) # enter max_iter=500 and it will take less time.
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

param_grid = {
    'activation': ['tanh','logistic','identity','relu'],
}
    
grid=GridSearchCV(nn,param_grid=param_grid,verbose=True,cv=cv)
grid_result=grid.fit(X_train,train_labels)



In [ ]:
grid_result.cv_results_

In [ ]:
grid_result.best_params_

In [ ]:
plt.bar(param_grid['activation'],grid_result.cv_results_['mean_test_score'])
plt.title('activation v/s Accuracy')
plt.xlabel('activation')
plt.ylabel('Accuracy')
plt.show()

## alpha

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold

nn=MLPClassifier(max_iter=500,early_stopping=True) # enter max_iter=500 and it will take less time.
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

param_grid = {

    'alpha': [0.0001, 0.05, 0.01,0.5,0.1],
    }
    
grid=GridSearchCV(nn,param_grid=param_grid,verbose=True,cv=cv)
grid_result=grid.fit(X_train,train_labels)



grid_result.best_params_


In [ ]:
grid_result.cv_results_

In [ ]:
plt.plot(param_grid['alpha'],grid_result.cv_results_['mean_test_score'])
plt.title('alpha v/s Accuracy')
plt.xlabel('alpha')
plt.ylabel('Accuracy')
plt.show()

## learning rate

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV,RepeatedStratifiedKFold

nn=MLPClassifier(max_iter=500,early_stopping=True) # enter max_iter=500 and it will take less time.
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)

param_grid = {
    'learning_rate': ['constant','adaptive'],
}
    
grid=GridSearchCV(nn,param_grid=param_grid,verbose=True,cv=cv)
grid_result=grid.fit(X_train,train_labels)



In [ ]:
grid_result.cv_results_

In [ ]:
grid_result.best_params_

In [ ]:
plt.bar(param_grid['learning_rate'],grid_result.cv_results_['mean_test_score'])
plt.title('learning_rate v/s Accuracy')
plt.xlabel('learning rate')
plt.ylabel('Accuracy')
plt.show()

## creating mlp model after tuning the hyper-parameters

In [151]:
from sklearn.neural_network import MLPClassifier
nn=MLPClassifier(hidden_layer_sizes=(200,200),max_iter=400,alpha=0.005,learning_rate='adaptive',activation='logistic',early_stopping=True) # enter max_iter=500 and it will take less time.

In [152]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
sss = KFold(n_splits=10,shuffle=True,random_state=0)

epochs=0
for train_index, val_index in sss.split(X_train, train_labels):
    #print("TRAIN:", train_index, "Val:", val_index)
    X_train_, X_val = X_train[train_index], X_train[val_index]
    y_train_, y_val = train_labels[train_index], train_labels[val_index]
    nn.fit(X_train_,y_train_)
    y_predicted=nn.predict(X_val)
    print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
    epochs+=1

Epochs #: 0 -----> val accuracy : 0.6896551724137931
Epochs #: 1 -----> val accuracy : 0.5172413793103449
Epochs #: 2 -----> val accuracy : 0.7209302325581395
Epochs #: 3 -----> val accuracy : 0.5813953488372093
Epochs #: 4 -----> val accuracy : 0.7674418604651163
Epochs #: 5 -----> val accuracy : 0.6395348837209303
Epochs #: 6 -----> val accuracy : 0.6046511627906976
Epochs #: 7 -----> val accuracy : 0.5465116279069767
Epochs #: 8 -----> val accuracy : 0.6511627906976745
Epochs #: 9 -----> val accuracy : 0.6744186046511628


In [153]:
final_ans_nn=nn.predict(X_test)

In [154]:
final_ans_nn

array([5, 0, 0, 3, 5, 0, 0, 2, 5, 1, 4, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 1, 0, 1, 0, 5, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 1, 0, 0, 5, 0,
       0, 0, 1, 7, 0, 0, 6, 0, 1, 0, 5, 0, 1, 4, 4, 0, 1, 0, 0, 3, 1, 0,
       4, 5, 0, 4, 0, 5, 0, 4, 0, 3, 1, 1, 0, 0, 3, 0, 1, 1, 0, 0, 2, 0,
       3, 2, 0, 0, 5, 0, 1, 3, 1, 3, 1, 1, 0, 0, 1, 0, 3, 4, 6, 0, 1, 1,
       5, 0, 3, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 1,
       0, 1, 4, 1, 1, 5, 1, 1, 0, 1, 0, 0, 2, 0, 5, 3, 6, 1, 1, 1, 0, 2,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 4, 0, 0, 3, 0, 5, 1, 6, 0,
       1, 3, 0, 2, 0, 6, 0, 1, 0, 3, 0, 1, 0, 6, 0, 0, 0, 0, 4, 1, 1, 0,
       0, 2, 2, 0, 2, 0, 2, 0, 0, 1, 4, 0, 0, 1, 1, 0, 1, 0, 5, 0, 0, 0,
       0, 5, 0, 1, 0, 2, 1, 0, 3, 0, 0, 5, 3, 1, 0, 0, 0, 0, 5, 6, 1, 1,
       2, 0, 0, 0, 6, 1, 1, 5, 0, 1, 0, 1, 3, 1, 0, 0, 3, 1, 0, 0, 0, 0,
       2, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 1, 0, 1, 1, 0, 0, 6, 0, 1,

In [139]:
ans_file_nn=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':final_ans_nn})

In [140]:
ans_file_nn.to_csv('/kaggle/working/submissions_tuned_nn.csv',index=False)

In [98]:
real_ans=[1,0,1,3,1,0,0,0,5,10,0,1,3,0,0,0,12,0,0,0,0,0,0,0,7,0,1,0,1,0,0,0,0,0,0,1,2,0,7,4,0,0,0,0,1,0,1,7,7,0,7,0,5,0,1,0,1,1,4,7,0,0,0,3,1,1,0,5,0,1,0,5,0,8,0,3,1,1,0,0,3,0,1,0,0,0,0,0,3,2,0,0,5,0,1,3,0,3,3,1,0,1,8,0,3,4,4,0,0,1,6,0,3,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,7,0,5,1,3,0,1,1,1,0,8,0,0,2,8,0,3,1,0,1,1,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,7,3,0,3,0,12,3,6,0,1,8,0,1,3,8,0,1,0,0,0,1,0,1,0,0,0,1,1,3,1,0,7,2,1,0,0,0,2,0,0,1,0,1,0,3,1,0,8,0,5,0,0,0,1,7,1,1,8,7,8,0,3,0,0,5,7,1,0,1,0,0,0,6,1,1,1,0,0,0,1,1,4,5,0,1,0,4,3,0,1,7,3,1,0,1,0,0,2,8,1,0,0,0,1,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,2,1,0,1,1,0,0,1,8,1,1,3,5,0,1,0,8,1,1,8,0,0,7,0,1,8,2,0,0,0,1,0,0,5,1,1,0,0,0,1,1,0,8,0,1,0,0,1,1,1,0,0,0,0,0,0,1,0,12,1,1,0,0,0,8,1,1,1,0,1,3,3,0,1,1,0,8,0,1,0,1,1,0,0,0,1,3,1,1,0,0]


In [155]:
counter=0
for i in range(len(real_ans)):
    if real_ans[i]==final_ans_nn[i]:
        counter+=1

In [156]:
print(counter)
print(counter/len(final_ans_nn))

246
0.6456692913385826


In [ ]:
ans

In [161]:
counter=0
for i in range(len(real_ans)):
    if ans[i]==real_ans[i]:
        counter+=1

In [162]:
print(counter)
print(counter/len(real_ans))

360
0.9448818897637795


In [163]:
counter=0
for i in range(len(real_ans)):
    if ans[i]==final_ans_nn[i]:
        counter+=1

In [164]:
print(counter)
print(counter/len(real_ans))

244
0.6404199475065617


In [97]:
now=np.array([1, 0, 0, 3, 1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 7, 0, 0, 0, 0, 5, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 5, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0,
       3, 2, 0, 0, 0, 0, 0, 3, 0, 3, 3, 1, 0, 0, 1, 0, 3, 0, 1, 0, 0, 0,
       6, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 5, 1, 3, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 5, 3, 6, 0,
       1, 0, 0, 0, 3, 6, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 1, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 6, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 5, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 0, 0])

In [99]:
counter=0
for i in range(len(real_ans)):
    if now[i]==real_ans[i]:
        counter+=1

In [100]:
print(counter)
print(counter/len(real_ans))

261
0.6850393700787402


In [106]:
counter=0
for i in range(len(final_ans_nn)):
    if now[i]==final_ans_nn[i]:
        counter+=1

In [107]:
print(counter)
print(counter/len(final_ans_nn))

237
0.6220472440944882


In [108]:
counter=0
for i in range(len(final_ans_nn)):
    if y_pred[i]==final_ans_nn[i]:
        counter+=1

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
print(counter)
print(counter/len(real_ans))

In [60]:
labels_df=pd.read_csv('../input/protein-localization/label_legend.csv')

In [61]:
all_labels=labels_df.to_dict()['Class']
#all_labels={all_labels[i]:i for i in all_labels}
all_labels

{0: 'nucleus',
 1: 'cytoplasm',
 2: 'mitochondria',
 3: 'cytoskeleton',
 4: 'plasma membrane',
 5: 'ER',
 6: 'golgi',
 7: 'vacuole',
 8: 'transport vesicles',
 9: 'peroxisome',
 10: 'endosome',
 11: 'integral membrane',
 12: 'extracellular',
 13: 'lipid particles',
 14: 'cell wall'}

## maybe use a nn here

In [62]:
Y_train_temp=pd.get_dummies(train_labels)
Y_train_temp

0   1   2   3   4   5   6   7   8   9   10  11  12  13  14
0     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
1     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4     1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
857   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0
858   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0
859   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0
860   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0
861   0   1   0   0   0   0   0   0   0   0   0   0   0   0   0

[862 rows x 15 columns]

In [130]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,SpatialDropout1D,LSTM
from keras.callbacks import EarlyStopping

model_nn = Sequential()
model_nn.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
#model_nn.add(Dense(128, activation='relu'))
model_nn.add(Dense(256,activation="relu"))
model_nn.add(Dense(15, activation='softmax'))

model_nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [131]:
model_nn.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 512)               1474048   
_________________________________________________________________
dense_41 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_42 (Dense)             (None, 15)                3855      
Total params: 1,609,231
Trainable params: 1,609,231
Non-trainable params: 0
_________________________________________________________________


In [132]:
model_nn.fit(X_train, Y_train_temp, epochs=60, batch_size=100,validation_split=0.2,shuffle=True,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/60
7/7 [==============================] - 1s 54ms/step - loss: 2.4729 - accuracy: 0.2661 - val_loss: 1.7385 - val_accuracy: 0.4509
Epoch 2/60
7/7 [==============================] - 0s 24ms/step - loss: 0.8480 - accuracy: 0.7633 - val_loss: 1.5470 - val_accuracy: 0.6012
Epoch 3/60
7/7 [==============================] - 0s 22ms/step - loss: 0.4427 - accuracy: 0.8986 - val_loss: 1.3772 - val_accuracy: 0.6127
Epoch 4/60
7/7 [==============================] - 0s 23ms/step - loss: 0.2235 - accuracy: 0.9448 - val_loss: 1.2980 - val_accuracy: 0.6069
Epoch 5/60
7/7 [==============================] - 0s 23ms/step - loss: 0.1456 - accuracy: 0.9627 - val_loss: 1.2817 - val_accuracy: 0.6301
Epoch 6/60
7/7 [==============================] - 0s 23ms/step - loss: 0.1042 - accuracy: 0.9756 - val_loss: 1.2671 - val_accuracy: 0.6358
Epoch 7/60
7/7 [==============================] - 0s 23ms/step - loss: 0.1040 - accuracy: 0.9660 - val_loss: 1.2720 - val_accuracy: 0.6243
Epoch 8/60
7/7 [===========

In [66]:
# from sklearn.model_selection import KFold
# from sklearn.metrics import accuracy_score
# sss = KFold(n_splits=10,shuffle=True,random_state=0)

# epochs=0
# for train_index, val_index in sss.split(X_train, train_labels):
#     #print("TRAIN:", train_index, "Val:", val_index)
#     X_train_, X_val = X_train[train_index], X_train[val_index]
#     y_train_, y_val = train_labels[train_index], train_labels[val_index]
#     model_nn.fit(X_train_,y_train_, epochs=150, batch_size=100)#,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
#     y_predicted=model_nn.predict(X_val)
#     print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
#     epochs+=1

In [91]:
y_pred=model_nn.predict(X_val)
#print(y_pred)
y_pred
#model_nn.evaluate(Y_val,y_pred)


array([[9.99993801e-01, 2.78721075e-07, 6.21542313e-07, ...,
        7.11577533e-11, 1.46475476e-09, 2.50574015e-08],
       [1.00000000e+00, 5.59428281e-10, 2.24981389e-09, ...,
        2.69315196e-13, 3.47903789e-11, 2.66071387e-10],
       [1.23618869e-04, 2.63719121e-05, 1.81322889e-07, ...,
        2.11945439e-09, 1.89837451e-07, 9.91981096e-07],
       ...,
       [8.69126916e-01, 2.79032513e-02, 8.97574127e-02, ...,
        3.02728199e-06, 7.19471936e-05, 9.00723080e-06],
       [1.33934736e-01, 2.99462322e-02, 1.17112875e-01, ...,
        4.95591084e-04, 5.59808174e-03, 2.52574473e-03],
       [1.06582493e-02, 8.61713886e-01, 2.67409207e-03, ...,
        1.13860995e-03, 3.35553777e-04, 1.91725860e-03]], dtype=float32)

In [92]:
ans=[]
labs=list(all_labels.keys())
for i in range(len(y_pred)):
    ans.append(labs[np.argmax(y_pred[i])])
#ans

[0,
 0,
 9,
 6,
 0,
 2,
 2,
 0,
 5,
 0,
 1,
 8,
 4,
 0,
 3,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 6,
 5,
 1,
 4,
 0,
 1,
 0,
 5,
 8,
 1,
 13,
 5,
 5,
 0,
 3,
 0,
 1,
 1,
 2,
 9,
 1,
 7,
 0,
 7,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 6,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 8,
 6,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 9,
 3,
 5,
 7,
 0,
 0,
 8,
 1]

In [80]:
accuracy_score(Y_val,ans)

NameError: name 'Y_val' is not defined

In [95]:
y_pred_ans=model_nn.predict(X_test)
#print(y_pred)
y_pred_ans
#model_nn.evaluate(Y_val,y_pred)


array([[6.71648741e-06, 9.55074906e-01, 1.01083820e-03, ...,
        1.61569915e-05, 1.10866549e-05, 4.14218766e-06],
       [9.99264181e-01, 1.42905410e-04, 4.96728935e-05, ...,
        4.05300149e-08, 5.01047271e-06, 6.74640614e-06],
       [3.12854141e-01, 3.11817944e-01, 5.32546937e-02, ...,
        9.97218187e-04, 5.36270300e-03, 2.58790841e-03],
       ...,
       [2.57451892e-01, 3.65097255e-01, 1.17130004e-01, ...,
        3.85777501e-04, 4.78267577e-03, 7.59127899e-04],
       [9.84715223e-01, 1.15020396e-02, 1.07167603e-03, ...,
        1.18232526e-06, 1.25047618e-05, 3.69908048e-06],
       [9.66982305e-01, 6.54193386e-03, 6.70605106e-03, ...,
        1.00534220e-04, 2.11055914e-04, 3.75187927e-04]], dtype=float32)

In [96]:
ans=[]
labs=list(all_labels.keys())
for i in range(len(y_pred_ans)):
    ans.append(labs[np.argmax(y_pred_ans[i])])
ans

[1,
 0,
 0,
 3,
 5,
 0,
 0,
 0,
 2,
 4,
 4,
 2,
 3,
 1,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 6,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 2,
 0,
 10,
 4,
 0,
 0,
 5,
 9,
 1,
 1,
 1,
 7,
 0,
 0,
 6,
 0,
 5,
 0,
 4,
 1,
 1,
 4,
 4,
 0,
 1,
 0,
 0,
 3,
 5,
 0,
 4,
 1,
 0,
 4,
 0,
 1,
 5,
 4,
 0,
 3,
 1,
 5,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 3,
 2,
 0,
 0,
 5,
 0,
 1,
 3,
 8,
 3,
 3,
 1,
 0,
 0,
 1,
 0,
 3,
 4,
 1,
 0,
 1,
 1,
 6,
 0,
 3,
 4,
 0,
 0,
 6,
 1,
 0,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 4,
 3,
 1,
 5,
 1,
 1,
 0,
 2,
 5,
 0,
 2,
 8,
 5,
 3,
 6,
 1,
 1,
 1,
 5,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 7,
 0,
 0,
 3,
 1,
 6,
 1,
 6,
 0,
 1,
 3,
 1,
 2,
 0,
 6,
 0,
 1,
 5,
 3,
 1,
 1,
 0,
 6,
 0,
 1,
 0,
 1,
 4,
 3,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 0,
 7,
 1,
 0,
 4,
 0,
 5,
 0,
 0,
 0,
 5,
 5,
 2,
 1,
 0,
 2,
 1,
 0,
 3,
 0,
 0,
 5,
 3,
 1,
 0,
 1,
 0,
 0,
 5,
 6,
 1,
 2,
 2,
 0,
 0,
 0,
 6,
 1,
 4,
 1,

In [110]:
counter=0
for i in range(len(real_ans)):
    if ans[i]==real_ans[i]:
        counter+=1

In [111]:
print(counter)
print(counter/len(real_ans))

240
0.6299212598425197


In [112]:
ans_file_nn=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':ans})

In [113]:
ans_file_nn.to_csv('/kaggle/working/submissions_tuned_nn.csv',index=False)

# may be we can try ensembling here

In [173]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
#clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
#clf2 = RandomForestClassifier(n_estimators=200, random_state=1)
#clf3 = GaussianNB()
#clf4= svm.SVC()
#X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
#y = np.array([1, 1, 1, 2, 2, 2])
eclf1 = VotingClassifier(estimators=[('rf', rnd_clf),('nn',nn )] , voting='soft')
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
sss = KFold(n_splits=10,shuffle=True,random_state=0)

epochs=0
for train_index, val_index in sss.split(X_train, train_labels):
    #print("TRAIN:", train_index, "Val:", val_index)
    X_train_, X_val = X_train[train_index], X_train[val_index]
    y_train_, y_val = train_labels[train_index], train_labels[val_index]
    eclf1.fit(X_train_,y_train_)
    y_predicted=eclf1.predict(X_val)
    print("Epochs #: {0} -----> val accuracy : {1}".format(epochs,accuracy_score(y_predicted,y_val)))
    epochs+=1
#eclf1 = eclf1.fit(X_train, Y_train)
y_val= eclf1.predict(X_test)
#accuracy_score(Y_val,y_val)

Epochs #: 0 -----> val accuracy : 0.7471264367816092
Epochs #: 1 -----> val accuracy : 0.7126436781609196
Epochs #: 2 -----> val accuracy : 0.7441860465116279
Epochs #: 3 -----> val accuracy : 0.6162790697674418
Epochs #: 4 -----> val accuracy : 0.7558139534883721
Epochs #: 5 -----> val accuracy : 0.7325581395348837
Epochs #: 6 -----> val accuracy : 0.6744186046511628
Epochs #: 7 -----> val accuracy : 0.5930232558139535
Epochs #: 8 -----> val accuracy : 0.6395348837209303
Epochs #: 9 -----> val accuracy : 0.7093023255813954


In [174]:
y_val

array([1, 0, 0, 3, 5, 0, 0, 0, 5, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 1, 7, 1, 0, 0, 0, 0,
       1, 0, 1, 7, 0, 0, 6, 0, 5, 0, 4, 0, 1, 1, 4, 0, 1, 0, 0, 3, 1, 1,
       4, 5, 0, 4, 0, 5, 0, 4, 0, 3, 1, 1, 0, 0, 3, 0, 1, 0, 0, 0, 2, 0,
       3, 2, 0, 0, 5, 0, 1, 3, 0, 3, 3, 1, 0, 0, 1, 0, 3, 4, 4, 0, 0, 1,
       5, 0, 3, 1, 0, 0, 6, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 3, 1, 5, 1, 1, 0, 1, 5, 0, 1, 8, 5, 3, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 7, 0, 0, 3, 0, 5, 1, 6, 0,
       1, 3, 0, 1, 3, 6, 0, 1, 0, 3, 0, 1, 0, 1, 0, 0, 0, 1, 4, 1, 1, 0,
       0, 2, 1, 0, 2, 0, 2, 0, 0, 1, 4, 0, 0, 3, 1, 0, 8, 0, 5, 0, 0, 0,
       1, 5, 1, 1, 8, 1, 8, 0, 3, 0, 0, 5, 1, 1, 0, 0, 0, 0, 5, 6, 1, 1,
       1, 0, 0, 0, 1, 1, 4, 5, 0, 1, 0, 1, 3, 1, 0, 1, 3, 1, 0, 0, 0, 0,
       2, 8, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0, 1, 6, 1, 1,

In [175]:
counter=0
for i in range(len(real_ans)):
    if y_val[i]==real_ans[i]:
        counter+=1

In [179]:
print(counter)
print(counter/len(real_ans))

301
0.7900262467191601


In [180]:
ans_file_nn=pd.DataFrame({'Key':test_df.iloc[:,0],'Label':y_val})

In [181]:
ans_file_nn.to_csv('/kaggle/working/submissions_tuned_nn_new.csv',index=False)

## Draw piecharts too